In [2]:
#Importing libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations, groupby
from collections import Counter

In [3]:
order_products_train_df = pd.read_csv("./data/order_products__train.csv")
order_products_prior_df = pd.read_csv("./data/order_products__prior.csv")
orders_df = pd.read_csv("./data/orders.csv")
products_df = pd.read_csv("./data/products.csv")
aisles_df = pd.read_csv("./data/aisles.csv")
departments_df = pd.read_csv("./data/departments.csv")

In [4]:
#orders_df[orders_df['eval_set']=='test'].head()
order_products_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
order_aisle_prod = pd.merge(order_products_prior_df,products_df,on='product_id',how='left')
order_aisle_prod = pd.merge(order_aisle_prod,aisles_df,on='aisle_id',how='left')
#order_products_prior_df = pd.merge(order_products_prior_df,orders_df,on='order_id',how='left')
order_aisle_prod.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle
0,2,33120,1,1,Organic Egg Whites,86,16,eggs
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients


In [8]:
## ADHOC AISLE 
df = order_aisle_prod.copy()
df['key'] = df['order_id'].astype(str) + '-' + df['aisle_id'].astype(str)
df = df.set_index('key')['product_id'].rename('item_id')
display(df.head(10))
type(df)
# df = order_aisle_prod.set_index('order_id')['product_id'].rename('item_id')
# display(df.head(10))
# type(df)

key
2-86     33120
2-83     28985
2-104     9327
2-19     45918
2-17     30035
2-83     17794
2-105    40141
2-88      1819
2-123    43668
3-120    33754
Name: item_id, dtype: int64

pandas.core.series.Series

In [20]:
# Converting dataframe with aisle_id as index and products 
df = order_aisle_prod.set_index('order_id')['product_id'].rename('item_id')
display(df.head(10))
type(df)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
Name: item_id, dtype: int64

pandas.core.series.Series

In [9]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]

In [10]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [11]:
%%time
rules = association_rules(df, 0.01)

Starting order_item:               32434489
Items with support >= 0.01:            2446
Remaining order_item:              22566243
Remaining orders with 2+ items:     3625741
Remaining order_item:               9813304


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


Item pairs:                          117933
Item pairs with support >= 0.01:       3053

Wall time: 2min 28s


In [12]:
item_name   = pd.read_csv('./data/products.csv')
item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)

In [13]:
rules_final

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,India Pale Ale,Beer,550,0.015169,1037,0.028601,1037,0.028601,0.530376,0.530376,18.543937
1,Beer,India Pale Ale,445,0.012273,1037,0.028601,1037,0.028601,0.429122,0.429122,15.003731
2,Crunchy Oats 'n Honey Granola Bars,Sweet & Salty Nut Granola Bars Peanut,582,0.016052,1297,0.035772,1249,0.034448,0.448728,0.465973,13.026188
3,Sweet & Salty Nut Granola Bars Peanut,Crunchy Oats 'n Honey Granola Bars,559,0.015418,1249,0.034448,1297,0.035772,0.447558,0.430995,12.511408
4,Peanut Butter Bar,Almond Butter,516,0.014232,1738,0.047935,1423,0.039247,0.296893,0.362614,7.564702
...,...,...,...,...,...,...,...,...,...,...,...
898,Organic Lemon,Banana,1769,0.048790,68581,1.891503,316134,8.719156,0.025794,0.005596,0.002958
877,Organic Navel Orange,Banana,946,0.026091,37474,1.033554,316134,8.719156,0.025244,0.002992,0.002895
2783,Strawberries,Organic Lemon,526,0.014507,104570,2.884100,68581,1.891503,0.005030,0.007670,0.002659
1716,Strawberries,Organic Strawberries,387,0.010674,104570,2.884100,208408,5.748011,0.003701,0.001857,0.000644


In [14]:
#test = rules_final[rules_final['lift'] > 1]
rules_final.to_csv("aisle_order_assoc.csv")